
# EGF Stability Testing — ready-to-run notebook

This notebook contains three testing modes for the polynomial generating functions of degree sequences of subgraphs (matchings-style supports):

- **Numerical (fast)**: randomized float evaluations in the upper half-plane (quick, may be subject to floating-point artifacts).
- **High-precision numeric**: evaluates numerically using SymPy `evalf(..., n=digits)` with many digits to reduce cancellation errors.
- **Symbolic (exact)**: builds the polynomial with exact rational coefficients and uses SymPy for exact manipulations/evaluations.

**How to use:** open the notebook, pick the cell labeled "Run configuration" and set the mode and parameters (m, n, max_size, trials, precision, etc.). Then run cells top-to-bottom (or the single run cell at the end).

The notebook will also avoid isomorphic duplicates (canonical set-system representation) and skip graphs with isolated vertices.



**Notes:**

- This notebook requires `sympy`, `networkx`, `numpy`, and `tqdm`. If they are not installed in your environment, run `pip install sympy networkx numpy tqdm` in a separate terminal or notebook cell.
- The notebook intentionally keeps things explicit and readable so you can inspect polynomials and failing evaluation points.


In [1]:

# Imports and basic helpers
import itertools, math, random
import networkx as nx
import numpy as np
from tqdm import tqdm
import sympy as sp

# Helper: factorial of a vector (python ints)
def vec_fact(v):
    from math import factorial
    return math.prod([factorial(int(x)) for x in v])

# Build P_G exactly as specified:
# P_G(x,y) = sum_{(alpha,beta) in J(G)} x^{m - alpha}/(m-alpha)! * y^{beta}/beta!
def build_P_G(G, m, n):
    # enumerate subgraphs (degree sequences)
    all_edges = list(G.edges())
    J = []
    for r in range(len(all_edges)+1):
        for subset in itertools.combinations(all_edges, r):
            sub = nx.Graph()
            sub.add_nodes_from(range(m+n))
            sub.add_edges_from(subset)
            alpha = [sub.degree(i) for i in range(m)]
            beta  = [sub.degree(m+j) for j in range(n)]
            if sum(alpha) == sum(beta):
                J.append((tuple(alpha), tuple(beta)))
    coeffs = {}
    for alpha, beta in J:
        ma = tuple(int(m - a) for a in alpha)
        beta = tuple(int(b) for b in beta)
        denom = vec_fact(ma) * vec_fact(beta)
        coeffs[(ma, beta)] = 1 / denom
    return coeffs

# Pretty-print polynomial (first few monomials)
def print_polynomial(coeffs, m, n, max_terms=20):
    terms = []
    for j, ((ma,beta), c) in enumerate(coeffs.items()):
        if j >= max_terms:
            terms.append("...")
            break
        x_part = "*".join([f"x{i}^{ma[i]}" for i in range(m) if ma[i] != 0])
        y_part = "*".join([f"y{j}^{beta[j]}" for j in range(n) if beta[j] != 0])
        term = f"{c:.6g}"
        if all(alpha == 0 for alpha in ma) and all(b == 0 for b in beta):
            term += "  <-- empty matching"
        if x_part:
            term += f" * {x_part}"
        if y_part:
            term += f" * {y_part}"
        terms.append(term)
    print(" +\n".join(terms))


In [2]:

# === Evaluation helpers ===

def eval_numeric_fast(coeffs, m, n, trials=200, tol=1e-12):
    """Fast randomized float tests. Returns (passed, witness_point_or_None)."""
    for _ in range(trials):
        x_vals = np.random.rand(m) + 1j*np.random.rand(m)
        y_vals = np.random.rand(n) + 1j*np.random.rand(n)
        val = 0+0j
        for (ma,beta), c in coeffs.items():
            xv = 1+0j
            for i in range(m):
                xv *= x_vals[i]**ma[i]
            yv = 1+0j
            for j in range(n):
                yv *= y_vals[j]**beta[j]
            val += c * xv * yv
        if abs(val) < tol:
            return False, (x_vals, y_vals, val)
    return True, None

def eval_high_precision(coeffs, m, n, trials=200, precision=50, tol=0):
    """High-precision numeric using SymPy evalf with 'precision' digits.
       Returns (passed, witness_point_or_None). If witness found val==0 at high precision.
"""
    x_syms = sp.symbols(f"x0:{m}") if m>0 else []
    y_syms = sp.symbols(f"y0:{n}") if n>0 else []
    P = sp.Integer(0)
    for (ma,beta), c in coeffs.items():
        term = sp.Rational(c)
        for i in range(m):
            term *= x_syms[i]**ma[i]
        for j in range(n):
            term *= y_syms[j]**beta[j]
        P += term
    for _ in range(trials):
        eval_point = {}
        for i in range(m):
            re = sp.Rational(random.randint(1,9), random.randint(1,3))
            im = sp.Rational(random.randint(1,9), random.randint(1,3))
            eval_point[x_syms[i]] = re + im*sp.I
        for j in range(n):
            re = sp.Rational(random.randint(1,9), random.randint(1,3))
            im = sp.Rational(random.randint(1,9), random.randint(1,3))
            eval_point[y_syms[j]] = re + im*sp.I
        val = P.evalf(subs=eval_point, n=precision)
        if val == 0:
            return False, (eval_point, val)
    return True, None

def eval_symbolic_exact(coeffs, m, n, test_points=None):
    """Symbolic exact evaluation at provided rational complex points (test_points list of dicts).
       If any exact evaluation equals zero, returns (False, witness_point, val)."""
    x_syms = sp.symbols(f"x0:{m}") if m>0 else []
    y_syms = sp.symbols(f"y0:{n}") if n>0 else []
    P = sp.Integer(0)
    for (ma,beta), c in coeffs.items():
        term = sp.Rational(c)
        for i in range(m):
            term *= x_syms[i]**ma[i]
        for j in range(n):
            term *= y_syms[j]**beta[j]
        P += term
    if not test_points:
        return True, None
    for pt in test_points:
        val = sp.simplify(P.subs(pt))
        if val == 0:
            return False, (pt, val)
    return True, None


In [3]:

# === Run configuration (edit before running) ===

mode = 'high_precision'  # choices: 'numeric', 'high_precision', 'symbolic'
max_size = 4             # test up to m=n=max_size
num_points = 300         # trials/points per polynomial
precision = 80           # digits for high-precision mode
stop_on_first_failure = True
verbose = False          # if True, print all processed canonical graphs
logfile = 'stability_log.txt'

print('Configuration ->', mode, 'max_size=', max_size, 'num_points=', num_points, 'precision=', precision)


Configuration -> high_precision max_size= 4 num_points= 300 precision= 80


In [ ]:

# === Main runner cell ===
from collections import defaultdict

def canonical_form_from_subset(subset, m, n):
    S = [set() for _ in range(n)]
    for u,v in subset:
        S[v].add(u)
    # canonical representation: sorted tuple of sorted lists
    return tuple(sorted(tuple(sorted(list(S[j]))) for j in range(n)))

# open log file
log = open(logfile, 'w')

for size in range(1, max_size+1):
    m = n = size
    print(f"\n=== Testing m=n={m} ===")
    all_edges = [(u, v) for u in range(m) for v in range(n)]
    seen = set()
    processed = 0
    total_canonical = 0
    # iterate by increasing number of edges (to find small edge failures first)
    for r in range(1, len(all_edges)+1):
        for subset in itertools.combinations(all_edges, r):
            cf = canonical_form_from_subset(subset, m, n)
            if cf in seen:
                continue
            seen.add(cf)
            # skip isolated vertices
            if any(len(list(s)) == 0 for s in cf):
                continue
            total_canonical += 1
            processed += 1
            # build graph and polynomial coeffs
            G = nx.Graph()
            G.add_nodes_from(range(m+n))
            G.add_edges_from([(u, m+v) for u,v in subset])
            coeffs = build_P_G(G, m, n)
            if verbose:
                print('\nGraph edges:', list(G.edges()))
                print('Set system:', ', '.join([f"S{k+1}={sorted(list(s))}" for k,s in enumerate(cf)]))
                print_polynomial(coeffs, m, n, max_terms=10)
            # choose mode
            if mode == 'numeric':
                passed, witness = eval_numeric_fast(coeffs, m, n, trials=num_points)
            elif mode == 'high_precision':
                passed, witness = eval_high_precision(coeffs, m, n, trials=num_points, precision=precision)
            elif mode == 'symbolic':
                # build a few rational test points for exact symbolic check
                test_pts = []
                x_syms = sp.symbols(f"x0:{m}") if m>0 else []
                y_syms = sp.symbols(f"y0:{n}") if n>0 else []
                for _ in range(num_points):
                    pt = {}
                    for i in range(m):
                        pt[x_syms[i]] = sp.Rational(random.randint(1,5), random.randint(1,3)) + sp.Rational(random.randint(1,5), random.randint(1,3))*sp.I
                    for j in range(n):
                        pt[y_syms[j]] = sp.Rational(random.randint(1,5), random.randint(1,3)) + sp.Rational(random.randint(1,5), random.randint(1,3))*sp.I
                    test_pts.append(pt)
                passed, witness = eval_symbolic_exact(coeffs, m, n, test_points=test_pts)
            else:
                raise ValueError('Unknown mode: ' + str(mode))
            if not passed:
                log.write(f"FAIL: m={m}, edges={len(subset)}, edges_list={list(G.edges())}\n")
                print('\n=== FAILURE detected ===')
                print('m,n =', m, ',', n)
                print('Edges:', list(G.edges()))
                print('Set system:', ', '.join([f"S{k+1}={sorted(list(s))}" for k,s in enumerate(cf)]))
                print('\nPolynomial (first few monomials):')
                print_polynomial(coeffs, m, n, max_terms=20)
                # print witness depending on mode
                if mode == 'numeric':
                    xvals, yvals, val = witness
                    print('\nNumeric witness (x,y,val):')
                    print(xvals, yvals, val)
                    log.write(f"Numeric witness: {xvals}, {yvals}, {val}\n")
                elif mode == 'high_precision':
                    pt, val = witness
                    print('\nHigh-precision witness (point, val):')
                    print(pt, val)
                    log.write(f"High-precision witness: {pt}, {val}\n")
                else:
                    pt, val = witness
                    print('\nSymbolic witness (exact point, val):')
                    print(pt, val)
                    log.write(f"Symbolic witness: {pt}, {val}\n")
                log.flush()
                if stop_on_first_failure:
                    log.close()
                    raise SystemExit('Stopped on first detected failure.')
            # end if not passed
    print(f"Processed {total_canonical} canonical graphs (m=n={m}).")

log.close()
print('\nDone. Results logged to', logfile)



=== Testing m=n=1 ===
Processed 1 canonical graphs (m=n=1).

=== Testing m=n=2 ===
Processed 6 canonical graphs (m=n=2).

=== Testing m=n=3 ===
Processed 84 canonical graphs (m=n=3).

=== Testing m=n=4 ===
